In [28]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [2]:
dhw_merge = pd.read_csv('dhw_merge.csv')
elec_merge  = pd.read_csv('elec_merge.csv')
n_elec_merge = pd.read_csv('n_elec_merge.csv')

dhw_merge = dhw_merge.drop(['YEAR'], axis=1)
elec_merge = elec_merge.drop(['YEAR'], axis=1)
n_elec_merge = n_elec_merge.drop(['YEAR'], axis=1)

##  For Scaling

In [3]:
dhw_X = dhw_merge.drop(['DHW'], axis=1)
dhw_Y = dhw_merge[['DHW']]

scaler_dhw = MinMaxScaler()
scaler_dhw.fit(dhw_X)
scaled_dhw_X = scaler_dhw.transform(dhw_X)

new_dhw_X = pd.DataFrame(scaled_dhw_X, index=dhw_X.index, columns=dhw_X.columns)
new_dhw = pd.concat([new_dhw_X, dhw_Y], axis=1)

In [4]:
elec_X = elec_merge.drop(['ELEC'], axis=1)
elec_Y = elec_merge[['ELEC']]

scaler_elec = MinMaxScaler()
scaler_elec.fit(elec_X)
scaled_elec_X = scaler_elec.transform(elec_X)

new_elec_X = pd.DataFrame(scaled_elec_X, index=elec_X.index, columns=elec_X.columns)
new_elec = pd.concat([new_elec_X, elec_Y], axis=1)

In [5]:
n_elec_X = n_elec_merge.drop(['n_elec'], axis=1)
n_elec_Y = n_elec_merge[['n_elec']]

scaler_n_elec = MinMaxScaler()
scaler_n_elec.fit(n_elec_X)
scaled_n_elec_X = scaler_n_elec.transform(n_elec_X)

new_n_elec_X = pd.DataFrame(scaled_n_elec_X, index=n_elec_X.index, columns=n_elec_X.columns)
new_n_elec = pd.concat([new_n_elec_X, n_elec_Y], axis=1)

In [6]:
input_dhw = new_dhw.iloc[:-1,]
target_dhw = new_dhw[['DHW']].iloc[1:]

trainX_dhw, testX_dhw, trainY_dhw, testY_dhw = train_test_split(input_dhw,target_dhw,test_size=0.3,shuffle=False,random_state=0)

In [7]:
input_elec = new_elec.iloc[:-1,]
target_elec = new_elec[['ELEC']].iloc[1:]

trainX_elec, testX_elec, trainY_elec, testY_elec = train_test_split(input_elec,target_elec,test_size=0.3,shuffle=False,random_state=0)

In [8]:
input = new_n_elec.iloc[:-1,]
target = new_n_elec[['n_elec']].iloc[1:]

trainX_n_elec, testX_n_elec, trainY_n_elec, testY_n_elec = train_test_split(input,target,test_size=0.3,shuffle=False,random_state=0)

## SVR

In [9]:
model_svr_dhw = SVR(kernel='rbf', gamma='auto')
model_svr_dhw.fit(trainX_dhw, trainY_dhw)

model_svr_elec = SVR(kernel='rbf', gamma='auto')
model_svr_elec.fit(trainX_elec, trainY_elec)

model_svr_n_elec = SVR(kernel='rbf', gamma='auto')
model_svr_n_elec.fit(trainX_n_elec, trainY_n_elec)

c:\Users\yunjae\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yunjae\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\yunjae\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(gamma='auto')

In [10]:
model_svr_pred_dhw = model_svr_dhw.predict(testX_dhw)
model_svr_preds_dhw = np.reshape(model_svr_pred_dhw,(-1,1))
svr_dhw_actual = testY_dhw.to_numpy()
svr_dhw_actual = np.reshape(svr_dhw_actual,(-1,1))

In [11]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(svr_dhw_actual, model_svr_preds_dhw))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(svr_dhw_actual, model_svr_preds_dhw))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(svr_dhw_actual, model_svr_preds_dhw))

mean_absolute_error :  0.054209611539572565
mean_squared_error :  0.008428053029079135
r2_score :  0.98766412228207


In [12]:
model_svr_pred_elec = model_svr_elec.predict(testX_elec)
model_svr_preds_elec = np.reshape(model_svr_pred_elec,(-1,1))
svr_elec_actual = testY_elec.to_numpy()
svr_elec_actual = np.reshape(svr_elec_actual,(-1,1))

In [13]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(svr_elec_actual, model_svr_preds_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(svr_elec_actual, model_svr_preds_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(svr_elec_actual, model_svr_preds_elec))

mean_absolute_error :  0.13437409937860412
mean_squared_error :  0.031189357473679513
r2_score :  0.4430593309012403


In [14]:
model_svr_pred_n_elec = model_svr_n_elec.predict(testX_n_elec)
model_svr_preds_n_elec = np.reshape(model_svr_pred_n_elec,(-1,1))
svr_n_elec_actual = testY_n_elec.to_numpy()
svr_n_elec_actual = np.reshape(svr_n_elec_actual,(-1,1))

In [15]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(svr_n_elec_actual, model_svr_preds_n_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(svr_n_elec_actual, model_svr_preds_n_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(svr_n_elec_actual, model_svr_preds_n_elec))

mean_absolute_error :  0.13265189992421086
mean_squared_error :  0.03634301018296824
r2_score :  0.9510622946205545


## XGBoost

In [16]:
xgb_model_dhw = xgb.XGBRegressor(n_estimators=200, learning_rate=0.01)
xgb_model_elec = xgb.XGBRegressor(n_estimators=200, learning_rate=0.01)
xgb_model_n_elec = xgb.XGBRegressor(n_estimators=200, learning_rate=0.01)

In [17]:
xgb_model_dhw.fit(trainX_dhw, trainY_dhw)
xgb_model_elec.fit(trainX_elec, trainY_elec)
xgb_model_n_elec.fit(trainX_n_elec, trainY_n_elec)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [18]:
xgb_model_pred_dhw = xgb_model_dhw.predict(testX_dhw)
xgb_model_preds_dhw = np.reshape(xgb_model_pred_dhw,(-1,1))
xgb_dhw_actual = testY_dhw.to_numpy()
xgb_dhw_actual = np.reshape(xgb_dhw_actual,(-1,1))

xgb_model_pred_elec = xgb_model_elec.predict(testX_elec)
xgb_model_preds_elec = np.reshape(xgb_model_pred_elec,(-1,1))
xgb_elec_actual = testY_elec.to_numpy()
xgb_elec_actual = np.reshape(xgb_elec_actual,(-1,1))

xgb_model_pred_n_elec = xgb_model_n_elec.predict(testX_n_elec)
xgb_model_preds_n_elec = np.reshape(xgb_model_pred_n_elec,(-1,1))
xgb_n_elec_actual = testY_n_elec.to_numpy()
xgb_n_elec_actual = np.reshape(xgb_n_elec_actual,(-1,1))

In [19]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(xgb_dhw_actual, xgb_model_preds_dhw))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(xgb_dhw_actual, xgb_model_preds_dhw))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(xgb_dhw_actual, xgb_model_preds_dhw))

mean_absolute_error :  0.10289827459518902
mean_squared_error :  0.019048124325899946
r2_score :  0.9721198559584877


In [20]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(xgb_elec_actual, xgb_model_preds_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(xgb_elec_actual, xgb_model_preds_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(xgb_elec_actual, xgb_model_preds_elec))

mean_absolute_error :  0.13934454790867734
mean_squared_error :  0.03276978340742732
r2_score :  0.414838054533321


In [22]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(xgb_n_elec_actual, xgb_model_preds_n_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(xgb_n_elec_actual, xgb_model_preds_n_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(xgb_n_elec_actual, xgb_model_preds_n_elec))

mean_absolute_error :  0.7681732757171676
mean_squared_error :  0.6308827682321209
r2_score :  0.15048437415398053


## LightGBM

In [23]:
model1= LGBMRegressor(
    learning_rate=0.01
    , n_estimators=200
)

model2= LGBMRegressor(
    learning_rate=0.01
    , n_estimators=200
)

model3= LGBMRegressor(
    learning_rate=0.01
    , n_estimators=200
)

lgbm_model_dhw = model1.fit(trainX_dhw, trainY_dhw, eval_metric='mae', verbose=0)
lgbm_model_elec = model2.fit(trainX_elec, trainY_elec, eval_metric='mae', verbose=0)
lgbm_model_n_elec = model3.fit(trainX_n_elec, trainY_n_elec, eval_metric='mae', verbose=0)

c:\Users\yunjae\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\yunjae\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\yunjae\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future releas

In [24]:
lgbm_model_pred_dhw = lgbm_model_dhw.predict(testX_dhw)
lgbm_model_preds_dhw = np.reshape(lgbm_model_pred_dhw,(-1,1))
lgbm_dhw_actual = testY_dhw.to_numpy()
lgbm_dhw_actual = np.reshape(lgbm_dhw_actual,(-1,1))

lgbm_model_pred_elec = lgbm_model_elec.predict(testX_elec)
lgbm_model_preds_elec = np.reshape(lgbm_model_pred_elec,(-1,1))
lgbm_elec_actual = testY_elec.to_numpy()
lgbm_elec_actual = np.reshape(lgbm_elec_actual,(-1,1))

lgbm_model_pred_n_elec = lgbm_model_n_elec.predict(testX_n_elec)
lgbm_model_preds_n_elec = np.reshape(lgbm_model_pred_n_elec,(-1,1))
lgbm_n_elec_actual = testY_n_elec.to_numpy()
lgbm_n_elec_actual = np.reshape(lgbm_n_elec_actual,(-1,1))

In [25]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(lgbm_dhw_actual, lgbm_model_preds_dhw))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(lgbm_dhw_actual, lgbm_model_preds_dhw))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(lgbm_dhw_actual, lgbm_model_preds_dhw))

mean_absolute_error :  0.1173839645637536
mean_squared_error :  0.020004148144311965
r2_score :  0.9707205537852968


In [26]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(lgbm_elec_actual, lgbm_model_preds_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(lgbm_elec_actual, lgbm_model_preds_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(lgbm_elec_actual, lgbm_model_preds_elec))

mean_absolute_error :  0.1344590962446814
mean_squared_error :  0.030568113575891478
r2_score :  0.45415273006470724


In [27]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(lgbm_n_elec_actual, lgbm_model_preds_n_elec))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(lgbm_n_elec_actual, lgbm_model_preds_n_elec))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(lgbm_n_elec_actual, lgbm_model_preds_n_elec))

mean_absolute_error :  0.15352190572629693
mean_squared_error :  0.04107443338337416
r2_score :  0.944691193453061


## Random Forest

In [29]:
rf_dhw = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=8, min_samples_split=8,n_estimators=200)

rf_elec = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=8, min_samples_split=8,n_estimators=200)

rf_n_elec = RandomForestRegressor(random_state=0, max_depth=5, min_samples_leaf=8, min_samples_split=8,n_estimators=200)

rf_dhw.fit(trainX_dhw, trainY_dhw)
rf_elec.fit(trainX_elec, trainY_elec)
rf_n_elec.fit(trainX_n_elec, trainY_n_elec)

C:\Users\yunjae\AppData\Local\Temp\ipykernel_15676\1979083861.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_dhw.fit(trainX_dhw, trainY_dhw)
C:\Users\yunjae\AppData\Local\Temp\ipykernel_15676\1979083861.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_elec.fit(trainX_elec, trainY_elec)
C:\Users\yunjae\AppData\Local\Temp\ipykernel_15676\1979083861.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_n_elec.fit(trainX_n_elec, trainY_n_elec)


RandomForestRegressor(max_depth=5, min_samples_leaf=8, min_samples_split=8,
                      n_estimators=200, random_state=0)

In [30]:
dhw_predict = rf_dhw.predict(testX_dhw)
elec_predict = rf_elec.predict(testX_elec)
n_elec_predict = rf_n_elec.predict(testX_n_elec)

dhw_actual = testY_dhw
elec_actual = testY_elec
n_elec_actual = testY_n_elec

In [31]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(dhw_actual, dhw_predict))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(dhw_actual, dhw_predict))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(dhw_actual, dhw_predict))

mean_absolute_error :  0.04180805234349381
mean_squared_error :  0.0080150805428414
r2_score :  0.9882685771987063


In [32]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(elec_actual, elec_predict))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(elec_actual, elec_predict))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(elec_actual, elec_predict))

mean_absolute_error :  0.12957898492955205
mean_squared_error :  0.029100711121363053
r2_score :  0.48035577402134855


In [33]:
from sklearn.metrics import mean_absolute_error
print('mean_absolute_error : ', mean_absolute_error(n_elec_actual, n_elec_predict))                                      

from sklearn.metrics import mean_squared_error
print('mean_squared_error : ', mean_squared_error(n_elec_actual, n_elec_predict))

from sklearn.metrics import r2_score   # model.score
print('r2_score : ', r2_score(n_elec_actual, n_elec_predict))

mean_absolute_error :  0.14910936371548567
mean_squared_error :  0.040995319826847974
r2_score :  0.9447977238670611
